### Notebook 2: Scraping Priors

The goal of this notebook is to write the function to scrape the `priors` and `basic_info` for each inmate currently in the Hunstville prison.

Final versions of these are saved in the `inmate_scrape.py`. 

It is noteworthy that significant enhancements have been made regarding `print statements` and `try:` and `except:` statements in the final function. Additionally, a key component of the final function is the automated `internal saving to .csv within the scrape`. This allowed me to work with partial data and build a shell of a notebook while the scrape function was running. 

It is also noteworthy that the csv files referred to in this notebook were not utilized for the project overall, and no longer exist. These acted as testers, and have not been saved. I eventually acquired over 40_000 observations of unique data, and this took almost 72 hours of running on AWS. 



In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from time import sleep

In [3]:
pd.set_option('display.max_columns', 150)

In [4]:
#I'm looking specifically at the inmates at Huntsville currently
#I want it to go through the list of names
#Find the href per name, and pull the data from the href link 

hunts_data_url = 'https://www.texastribune.org/library/data/texas-prisons/units/huntsville/'

res = requests.get(hunts_data_url)

soup = BeautifulSoup(res.content)

### Trying to get it done with one Name, and then will create a function to do it for all of them.

In [5]:
#body of page 1
tbody_1 = soup.find('tbody')

In [6]:
#how to get the link for one page (will need to add base_url (below))
link = tbody_1.find({'a', 'href'}).attrs

In [7]:
link #creates dictionary, can pull value

{'href': '/library/data/texas-prisons/inmates/carlos-stiff/1196290/'}

In [8]:
link.values()

dict_values(['/library/data/texas-prisons/inmates/carlos-stiff/1196290/'])

## Trying to create list of slugs to pull from the names of inmates currently in Hunstville state prison.

In [9]:
base_url = 'https://www.texastribune.org'

In [10]:
#have to make list of slugs to run through to pull 'href' link
list_of_slugs = []
for i in range(1,49):
    slug = '?page='+str(i)
    
    list_of_slugs.append(slug)
    
list_of_slugs[:3]

['?page=1', '?page=2', '?page=3']

In [13]:
all_prisons_main = pd.read_csv('../datasets/all_prisons_main.csv')

In [17]:
all_prisons_main.head(2)

,Unnamed: 0,name,age,main_crime,entered_on,term,crime_location,home_county,prison_name
0,0,Micah Troy Duncan,32,CAPITAL MURDER PERSON U/SIX YR,12/22/2011,Life Without Parole,Grayson,Grayson,Allred
1,1,Heyman Harold Guillory,44,CAPITAL MURDER,3/16/2011,Life Without Parole,Jefferson,NaN,Allred


Working to pull in the links for each of the names.

In [24]:
#rerun this once i get links
all_prisons_main.to_csv('../datasets/my_data/ap_with_plinks.csv')

In [33]:
unique_plinks = all_prisons_main['prison_links'].unique()

In [ ]:
dict_list = []

for slug in list_of_slugs: 
    sleep(1)
    res = requests.get(url + slug)
    soup = BeautifulSoup(res.content)

    print(' ')
    print(slug)

    tbody = soup.find('tbody')

    for row in tbody.find_all('tr'):
        name = row.find('td', {'data-title': 'Name'})

        link = name.find({'a', 'href'}).attrs

        print(link)

        dict_list.append(link)

print('Done.')

In [11]:
def scrape_for_hrefs(url, list_of_slugs):    
    dict_list = []

    for slug in list_of_slugs: 
        sleep(1)
        res = requests.get(url + slug)
        soup = BeautifulSoup(res.content)

        print(' ')
        print(slug)

        tbody = soup.find('tbody')

        for row in tbody.find_all('tr'):
            name = row.find('td', {'data-title': 'Name'})
            
            link = name.find({'a', 'href'}).attrs
            
            print(link)
            
            dict_list.append(link)

    print('Done.')
    
    return dict_list

In [6]:
def scrape_for_hrefs_all(url, list_of_slugs):    
    dict_list = []

    for slug in list_of_slugs: 
        try:
            sleep(1)
            res = requests.get(url + slug)
            soup = BeautifulSoup(res.content)

            print(' ')
            print(slug)

            tbody = soup.find('tbody')

            for row in tbody.find_all('tr'):
                name = row.find('td', {'data-title': 'Name'})

                link = name.find({'a', 'href'}).attrs

                print(link)

                dict_list.append(link)
        except:
            print('No more pages.')
            break

    print('Done.')

    return dict_list

The next cell was run in the initial code. Now that the .csv has been created, these are here to show what was done, but do not need to be run again. 

The steps taken below were to:
- use `scrape_for_hrefs()` function to generate a list of slugs
- add the list of slugs as a new column into the dataframe
- drop column = 'Unnamed: 0'
- `create csv` of dataframe with links added

Bringing in the updated csv.

In [16]:
df = pd.read_csv('../datasets/gooddtypes_links.csv')
df.drop(columns = 'Unnamed: 0', inplace = True)

df.head()

,name,age,main_crime,entered_on,term,crime_location,home_county,term_rd2,links
0,Carlos Stiff,31,CAPITAL MURDER,2011-12-16,Life Without Parole,Gregg,Gregg,500.0,/library/data/texas-prisons/inmates/carlos-sti...
1,Winfred Earl Lauderdale,42,CAPITAL MURDER,2000-05-05,Life,Bexar,Bexar,150.0,/library/data/texas-prisons/inmates/winfred-ea...
2,Jerry Marcus Jernigan,69,AGGRV ROBB W/DEAD WPN,1984-03-20,Life,Tarrant,Dallas,150.0,/library/data/texas-prisons/inmates/jerry-marc...
3,Larry Dale Barker,44,CAPITAL MURDER,2012-03-15,Life,Galveston,Galveston,150.0,/library/data/texas-prisons/inmates/larry-dale...
4,David Randal Lacombe,54,MURDER W/DEADLY WPN,1992-03-19,Life,Dallas,Dallas,150.0,/library/data/texas-prisons/inmates/david-rand...


In [19]:
df_dtypes.dtypes

Unnamed: 0          int64
name               object
age                 int64
main_crime         object
entered_on         object
term               object
crime_location     object
home_county        object
term_rd2          float64
dtype: object

In [20]:
#manually updating datetime, could not save it within the .csv this way 
df['entered_on'] = pd.to_datetime(df['entered_on'])

In [21]:
df.dtypes

name                      object
age                        int64
main_crime                object
entered_on        datetime64[ns]
term                      object
crime_location            object
home_county               object
term_rd2                 float64
links                     object
dtype: object

In [34]:
#pulling out the slugs to use in a function
individuals_slugs_list = df['links']

#### Pulling data from one individual, including priors, to test/build out function

In [200]:
ind_url = 'https://www.texastribune.org/library/data/texas-prisons/inmates/anthony-ray-moore/1136437/'

res_ind = requests.get(ind_url)
soup_ind = BeautifulSoup(res_ind.content)

In [201]:
tbody = soup_ind.find('tbody')
div = soup_ind.find_all('div', {'class': 'cell w-6'})


Acquiring sex, race, hair color, eye color, projected release date, and TDCJ ID for an individual.

In [202]:
div[0].find_all('td')

[<td>25 years</td>,
 <td>3/6/2024</td>,
 <td>00687022</td>,
 <td><a href="/library/data/texas-prisons/units/huntsville/">Huntsville</a></td>,
 <td>7/25/1961</td>,
 <td><a href="/library/data/texas-prisons/counties/harris/home/">Harris</a></td>]

In [203]:
div[1].find_all('td')[2].text
#store as list 
#call the index you want
#df['sex'] = index[0]

'Black'

In [204]:
div[1].find_all('th')

[<th scope="row">Sex</th>,
 <th scope="row">Age</th>,
 <th scope="row">Race</th>,
 <th scope="row">Height</th>,
 <th scope="row">Weight</th>,
 <th scope="row">Hair Color</th>]

Finding their names for the dataframe.

In [205]:
header = soup_ind.find('header', {'class': 'auxiliary'})

In [206]:
header.find('h1').text

'Anthony Ray Moore'

#### Pulling in an inmates prior crimes (if they exist):
- what they were
- when they were committed
- sentence length
- when sentence began

In [182]:
td = soup_ind.find_all('td')

In [183]:
tbody = soup_ind.find('tbody')

In [184]:
tbody

<tbody>
<tr class="odd">
<td data-title="Crime">DEL OF COCAINE</td>
<td data-title="Committed On">7/18/2000</td>
<td data-title="County"><a href="/library/data/texas-prisons/counties/harris/crime/">Harris</a></td>
<td data-title="Term">2 years</td>
<td data-title="Sentence Began">7/18/2000</td>
</tr>
<tr class="even">
<td data-title="Crime">POSS OF COCAINE</td>
<td data-title="Committed On">7/18/2000</td>
<td data-title="County"><a href="/library/data/texas-prisons/counties/harris/crime/">Harris</a></td>
<td data-title="Term">2 years</td>
<td data-title="Sentence Began">7/18/2000</td>
</tr>
<tr class="odd">
<td data-title="Crime">BURG OF A MTR VEH WIC/THEFT</td>
<td data-title="Committed On">9/13/1993</td>
<td data-title="County"><a href="/library/data/texas-prisons/counties/harris/crime/">Harris</a></td>
<td data-title="Term">25 years</td>
<td data-title="Sentence Began">9/13/1993</td>
</tr>
<tr class="even">
<td data-title="Crime">BURG MTR VEH W/I THEFT</td>
<td data-title="Committed

In [185]:
for row in tbody.find_all('tr'):
    print(row.find('td', {'data-title': 'Committed On'}).text)

7/18/2000
7/18/2000
9/13/1993
10/13/1990


In [186]:
for row in tbody.find_all('tr'):
    print(row.find('td').text)

DEL OF COCAINE
POSS OF COCAINE
BURG OF A MTR VEH WIC/THEFT
BURG MTR VEH W/I THEFT


In [187]:
crimes = tbody.find_all('td', {'data-title': 'Crime'})

for crime in crimes:
    print(crime.text)

DEL OF COCAINE
POSS OF COCAINE
BURG OF A MTR VEH WIC/THEFT
BURG MTR VEH W/I THEFT


In [189]:
priors_all_info = tbody.find_all('td')

priors_all_info[::5]

[<td data-title="Crime">DEL OF COCAINE</td>,
 <td data-title="Crime">POSS OF COCAINE</td>,
 <td data-title="Crime">BURG OF A MTR VEH WIC/THEFT</td>,
 <td data-title="Crime">BURG MTR VEH W/I THEFT</td>]

In [190]:
tbody.find_all('td', {'data-title': 'Committed On'})

[<td data-title="Committed On">7/18/2000</td>,
 <td data-title="Committed On">7/18/2000</td>,
 <td data-title="Committed On">9/13/1993</td>,
 <td data-title="Committed On">10/13/1990</td>]

When I come back to it - trying to run something in the function that populates data['crime[i]] = 

## Creating a function to pull data from all of the individual inmate information pages

In [192]:
[slug for slug in individuals_slugs_list[398:402]]

['/library/data/texas-prisons/inmates/john-stanley-kramer/1123433/',
 '/library/data/texas-prisons/inmates/anthony-ray-moore/1136437/',
 '/library/data/texas-prisons/inmates/charles-carter-edmondson/1091981/',
 '/library/data/texas-prisons/inmates/david-ford/1112027/']

In [199]:
base_url + individuals_slugs_list[399]

'https://www.texastribune.org/library/data/texas-prisons/inmates/anthony-ray-moore/1136437/'

**Below is my `FINAL FUNCTION`. Do Not Manipulate or Alter.**

Want to come and create a `.py` file for this function and to pull it in to the code. It's too big for the page I feel like...

In [268]:
def individual_scrape_incl_priors(base_url, list_of_slugs):
    dict_list = []

    counter = 1
    for slug in list_of_slugs:
        sleep(1)
        res = requests.get(base_url + slug)
        soup = BeautifulSoup(res.content)

        print(' ')
        print(f'slug # {counter}')
        print(slug)
        
        counter += 1
        
        #setting up what i need to access 
        tbody = soup.find('tbody')
        div = soup.find_all('div', {'class': 'cell w-6'})

        
        #finding the individual's name
        header = soup.find('header', {'class': 'auxiliary'})
        name = header.find('h1').text

        div = soup.find_all('div', {'class': 'cell w-6'})

        #div[1] and div[0] is the section with the info I want for ind info
        columns_names = div[1].find_all('th')
        individual_information = div[1].find_all('td')
        more_ind_inf = div[0].find_all('td')
        
        
        #getting into the priors
        priors_all_info = tbody.find_all('td')
        crimes = tbody.find_all('td', {'data-title': 'Crime'})
        committed_date = tbody.find_all('td', {'data-title': 'Committed On'})
        term_of_prior = tbody.find_all('td', {'data-title': 'Term'})
        sentence_begins = tbody.find_all('td', {'data-title': 'Sentence Began'})
        
         
        
        
        #creating the dictionary to make into a dataframe
        data = {}
        data['name'] = name
        
        #adding 'sex' column
        try:
            data['sex'] = individual_information[0].text
        except:
            data['sex'] = None
            
        #adding 'race' column
        try:
            data['race'] = individual_information[2].text
        except: 
            data['race'] = individual_information[2].text
        
        #adding 'height' column
        try:
            data['height'] = individual_information[3].text
        except:
            data['height'] = None
            
            
        #adding 'weight' column
        try:
            data['weight'] = individual_information[4].text
        except:
            data['weight'] = None
        
        #adding 'hair color' column
        try:
            data['hair_color'] = individual_information[5].text
        except:
            data['hair_color'] = None
        
        #THIS WOULD BREAK OTHERWISE
        try:
            data['eye_color'] = individual_information[6].text
            
        except:
            data['eye_color'] = None #slug 399 (AR MOORE) does not have eye-color
        
        data['TDCJ_ID'] = more_ind_inf[2].text
        data['proj_release_date'] = more_ind_inf[1].text
        
        
    
        #creating a for loop with the length
        for num in range(len(crimes)):
                
    
            data['pr_crime_' + str(num)] = crimes[num].text
            data['pr_commit_date_' + str(num)] = committed_date[num].text
            data['pr_term_' + str(num)] = term_of_prior[num].text
            data['pr_begins_' + str(num)] = sentence_begins[num].text
            
            print(f'ind prior {num} done.')
        

        dict_list.append(data)

            
        print('And another..')


    #transforming the dictionary to a dataframe
        df = pd.DataFrame(dict_list)
    
    #organizing the columns in the order I want 


    print('Done.')

    return df

In [269]:
df_priors = individual_scrape_incl_priors(base_url, individuals_slugs_list)

 
slug # 1
/library/data/texas-prisons/inmates/carlos-stiff/1196290/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
And another..
 
slug # 2
/library/data/texas-prisons/inmates/winfred-earl-lauderdale/1128958/
ind prior 0 done.
And another..
 
slug # 3
/library/data/texas-prisons/inmates/jerry-marcus-jernigan/1148737/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
And another..
 
slug # 4
/library/data/texas-prisons/inmates/larry-dale-barker/1092116/
ind prior 0 done.
And another..
 
slug # 5
/library/data/texas-prisons/inmates/david-randal-lacombe/1124227/
ind prior 0 done.
And another..
 
slug # 6
/library/data/texas-prisons/inmates/larry-dell-davis/1070979/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
And another..
 
slug # 7
/library/data/texas-prisons/inmates/johnny-wayne-hill/1123994/
ind prior 0 done.
ind prior 1 done.
And another..
 
slug # 8
/library/data/texas-prisons/inmates/miguel-teran-bueno-sr/1067625/
ind prior 0 done.
ind prior 1 done.
And another

 
slug # 62
/library/data/texas-prisons/inmates/chadrick-b-pate/1174223/
ind prior 0 done.
ind prior 1 done.
And another..
 
slug # 63
/library/data/texas-prisons/inmates/juan-manuel-gonzalez/1090944/
ind prior 0 done.
And another..
 
slug # 64
/library/data/texas-prisons/inmates/richard-brian-devereaux/1079962/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
ind prior 3 done.
ind prior 4 done.
And another..
 
slug # 65
/library/data/texas-prisons/inmates/victor-edward-hornal/1129444/
ind prior 0 done.
And another..
 
slug # 66
/library/data/texas-prisons/inmates/henry-cabrera/1074180/
ind prior 0 done.
And another..
 
slug # 67
/library/data/texas-prisons/inmates/darryl-glenn-holman/1128361/
ind prior 0 done.
And another..
 
slug # 68
/library/data/texas-prisons/inmates/russell-wayne-mcsland/1172717/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
ind prior 3 done.
And another..
 
slug # 69
/library/data/texas-prisons/inmates/thomas-ray-steele/1194678/
ind prior 0 done.
in

 
slug # 121
/library/data/texas-prisons/inmates/scotty-lynn-collum/1104644/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
And another..
 
slug # 122
/library/data/texas-prisons/inmates/michael-wayne-cantu/1080970/
ind prior 0 done.
And another..
 
slug # 123
/library/data/texas-prisons/inmates/rey-lewis-sanchez/1181974/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
ind prior 3 done.
ind prior 4 done.
And another..
 
slug # 124
/library/data/texas-prisons/inmates/james-allen-hamm/1110397/
ind prior 0 done.
And another..
 
slug # 125
/library/data/texas-prisons/inmates/jeffery-lee-manns/1152001/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
ind prior 3 done.
ind prior 4 done.
And another..
 
slug # 126
/library/data/texas-prisons/inmates/jason-michael-williams/1200784/
ind prior 0 done.
And another..
 
slug # 127
/library/data/texas-prisons/inmates/jason-edward-hernandez/1124439/
ind prior 0 done.
And another..
 
slug # 128
/library/data/texas-prisons/inmates/pau

 
slug # 178
/library/data/texas-prisons/inmates/johnnie-davis/1070899/
ind prior 0 done.
ind prior 1 done.
And another..
 
slug # 179
/library/data/texas-prisons/inmates/tony-lee-batt/1094263/
ind prior 0 done.
ind prior 1 done.
And another..
 
slug # 180
/library/data/texas-prisons/inmates/george-harold-ballew/1089766/
ind prior 0 done.
And another..
 
slug # 181
/library/data/texas-prisons/inmates/jose-luis-rubio/1171996/
ind prior 0 done.
ind prior 1 done.
And another..
 
slug # 182
/library/data/texas-prisons/inmates/jerrald-dewayne-allison/1073605/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
ind prior 3 done.
ind prior 4 done.
ind prior 5 done.
And another..
 
slug # 183
/library/data/texas-prisons/inmates/daniel-almendarez-yvanez/1207319/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
ind prior 3 done.
ind prior 4 done.
And another..
 
slug # 184
/library/data/texas-prisons/inmates/steven-kelley-lawson/1129663/
ind prior 0 done.
ind prior 1 done.
ind prior 2 don

 
slug # 232
/library/data/texas-prisons/inmates/andrea-lamont-booker/1110370/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
ind prior 3 done.
And another..
 
slug # 233
/library/data/texas-prisons/inmates/ronald-eugene-hill/1224541/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
ind prior 3 done.
And another..
 
slug # 234
/library/data/texas-prisons/inmates/justin-dwayne-music/1148394/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
And another..
 
slug # 235
/library/data/texas-prisons/inmates/omar-alexander-rodriguez/1165746/
ind prior 0 done.
And another..
 
slug # 236
/library/data/texas-prisons/inmates/leonard-darnell-williams/1201643/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
ind prior 3 done.
And another..
 
slug # 237
/library/data/texas-prisons/inmates/rudolfo-hernandez-lopez/1143386/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
ind prior 3 done.
And another..
 
slug # 238
/library/data/texas-prisons/inmates/andre-bertrand/1103997/
i

...

 
slug # 1175
/library/data/texas-prisons/inmates/quinten-leon-robinson/1161601/
ind prior 0 done.
ind prior 1 done.
And another..
 
slug # 1176
/library/data/texas-prisons/inmates/jose-ramirez-jr/1139131/
ind prior 0 done.
ind prior 1 done.
And another..
 
slug # 1177
/library/data/texas-prisons/inmates/guadalupe-azoca-gonzalez/1090750/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
ind prior 3 done.
And another..
 
slug # 1178
/library/data/texas-prisons/inmates/ricardo-bustos/1072623/
ind prior 0 done.
And another..
 
slug # 1179
/library/data/texas-prisons/inmates/james-preston-richardson/1152804/
ind prior 0 done.
And another..
 
slug # 1180
/library/data/texas-prisons/inmates/terry-elijah-meister/1174999/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
ind prior 3 done.
ind prior 4 done.
ind prior 5 done.
ind prior 6 done.
ind prior 7 done.
ind prior 8 done.
ind prior 9 done.
ind prior 10 done.
ind prior 11 done.
ind prior 12 done.
ind prior 13 done.
ind prior 14 don

 
slug # 1230
/library/data/texas-prisons/inmates/brion-joseph-white/1196644/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
And another..
 
slug # 1231
/library/data/texas-prisons/inmates/stanley-ray-stephens/1195232/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
ind prior 3 done.
ind prior 4 done.
ind prior 5 done.
ind prior 6 done.
ind prior 7 done.
And another..
 
slug # 1232
/library/data/texas-prisons/inmates/amaury-javier-laboy-rosado/1168909/
ind prior 0 done.
ind prior 1 done.
And another..
 
slug # 1233
/library/data/texas-prisons/inmates/mario-santiago-rodriguez/1165662/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
And another..
 
slug # 1234
/library/data/texas-prisons/inmates/justin-clay-walker/1191402/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
ind prior 3 done.
ind prior 4 done.
ind prior 5 done.
ind prior 6 done.
ind prior 7 done.
ind prior 8 done.
And another..
 
slug # 1235
/library/data/texas-prisons/inmates/samuel-rodriguez-martine

 
slug # 1283
/library/data/texas-prisons/inmates/mauricio-a-buendia/1067608/
ind prior 0 done.
And another..
 
slug # 1284
/library/data/texas-prisons/inmates/chermorne-earl-calloway/1077068/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
And another..
 
slug # 1285
/library/data/texas-prisons/inmates/william-deshayes/1236768/
ind prior 0 done.
And another..
 
slug # 1286
/library/data/texas-prisons/inmates/kerry-franco/1115352/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
And another..
 
slug # 1287
/library/data/texas-prisons/inmates/roy-davis/1071915/
ind prior 0 done.
And another..
 
slug # 1288
/library/data/texas-prisons/inmates/george-ray-gutierrez-sr/1105483/
ind prior 0 done.
And another..
 
slug # 1289
/library/data/texas-prisons/inmates/kristopher-lee-rocco/1162434/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
ind prior 3 done.
ind prior 4 done.
ind prior 5 done.
ind prior 6 done.
And another..
 
slug # 1290
/library/data/texas-prisons/inmates/roy-j

 
slug # 1336
/library/data/texas-prisons/inmates/johnny-mack-yarbrough/1206638/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
ind prior 3 done.
ind prior 4 done.
ind prior 5 done.
And another..
 
slug # 1337
/library/data/texas-prisons/inmates/charles-ray-pless-ii/1130064/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
ind prior 3 done.
And another..
 
slug # 1338
/library/data/texas-prisons/inmates/jason-benitez-luviano/1243544/
ind prior 0 done.
ind prior 1 done.
And another..
 
slug # 1339
/library/data/texas-prisons/inmates/donald-herbert-grant/1242591/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
And another..
 
slug # 1340
/library/data/texas-prisons/inmates/larry-delgado/1077516/
ind prior 0 done.
And another..
 
slug # 1341
/library/data/texas-prisons/inmates/roger-dale-shoemaker/1251433/
ind prior 0 done.
ind prior 1 done.
ind prior 2 done.
ind prior 3 done.
And another..
 
slug # 1342
/library/data/texas-prisons/inmates/daterrius-alexander/1247203/
in

 
slug # 1392
/library/data/texas-prisons/inmates/brendon-cristobal/1248128/
ind prior 0 done.
ind prior 1 done.
And another..
 
slug # 1393
/library/data/texas-prisons/inmates/terrell-morris/1215654/
ind prior 0 done.
And another..
 
slug # 1394
/library/data/texas-prisons/inmates/javoni-marquel-green/1209087/
ind prior 0 done.
And another..
 
slug # 1395
/library/data/texas-prisons/inmates/ruben-garcia/1237230/
ind prior 0 done.
And another..
 
slug # 1396
/library/data/texas-prisons/inmates/cosme-bejaran/1241160/
ind prior 0 done.
ind prior 1 done.
And another..
 
slug # 1397
/library/data/texas-prisons/inmates/ryan-shane-casas/1213802/
ind prior 0 done.
ind prior 1 done.
And another..
 
slug # 1398
/library/data/texas-prisons/inmates/christopher-coker/1102199/
ind prior 0 done.
And another..
 
slug # 1399
/library/data/texas-prisons/inmates/keith-brandon-griffin/1223927/
ind prior 0 done.
And another..
 
slug # 1400
/library/data/texas-prisons/inmates/ivan-jay-tucker/1179166/
ind p

In [270]:
df_priors.shape

(1411, 205)

In [273]:
len(df_priors[df_priors.duplicated()])

160

In [284]:
[slug for slug in individuals_slugs_list if 'larry' in slug]

['/library/data/texas-prisons/inmates/larry-dale-barker/1092116/',
 '/library/data/texas-prisons/inmates/larry-dell-davis/1070979/',
 '/library/data/texas-prisons/inmates/larry-dale-barker/1092116/',
 '/library/data/texas-prisons/inmates/larry-jackson/1142983/',
 '/library/data/texas-prisons/inmates/larry-leroy-white/1197139/',
 '/library/data/texas-prisons/inmates/larry-weldon-harvey/1115832/',
 '/library/data/texas-prisons/inmates/larry-onell-lathan/1128916/',
 '/library/data/texas-prisons/inmates/larry-edward-talbott/1198922/',
 '/library/data/texas-prisons/inmates/larry-lynn-henry/1232498/',
 '/library/data/texas-prisons/inmates/larry-delgado/1077516/']

In [285]:
df_priors.drop_duplicates(inplace=True)

In [287]:
df_priors.to_csv('../datasets/df_priors.csv')

**The function below is a BASE FUNCTION. `DO NOT DELETE OR MANIPULATE.`**

In [288]:
def individual_scrape_basic(base_url, list_of_slugs):
    dict_list = []

    counter = 1
    for slug in list_of_slugs[:3]:
        sleep(1)
        res = requests.get(base_url + slug)
        soup = BeautifulSoup(res.content)

        print(' ')
        print(f'slug # {counter}')
        print(slug)
        
        counter += 1
        
        tbody = soup_ind.find('tbody')
        div = soup_ind.find_all('div', {'class': 'cell w-6'})

        header = soup.find('header', {'class': 'auxiliary'})
        name = header.find('h1').text

        div = soup.find_all('div', {'class': 'cell w-6'})

        #div[1] is the section with the info I want
        #calling all of them into a list rather than a for loop
        columns_names = div[1].find_all('th')
        individual_information = div[1].find_all('td')
        more_ind_inf = div[0].find_all('td')
        
        
        #creating the dictionary to make into a dataframe
        data = {}
        data['name'] = name
        data['sex'] = individual_information[0].text
        data['race'] = individual_information[2].text
        data['height'] = individual_information[3].text
        data['weight'] = individual_information[4].text
        data['hair_color'] = individual_information[5].text
        data['eye_color'] = individual_information[6].text
        data['TDCJ_ID'] = more_ind_inf[2].text
        data['proj_release_date'] = more_ind_inf[1].text
        

        dict_list.append(data)

            
        print('And another..')


    #transforming the dictionary to a dataframe
    df = pd.DataFrame(dict_list)
    
    #organizing the columns in the order I want 
    df_final = df[['name','TDCJ_ID', 'sex', 'race', 'height',
                   'weight', 'eye_color', 'hair_color']]

    print('Done.')

    return df_final